**TODO ADD Jekyll header**

In this post, I will visualize the simulated hypothetical spread of Covid-19 in neighbourhoods around Schiphol in The Netherlands, within a 30 km radius. The data was generated in [this previous post](https://jvlanalytics.nl/covid-19-simulation). This new post is mostly an excuse to play around a bit with Kepler.gl, because the data was simulated using not entirely realistic assumptions. Nevertheless, I'm hoping to gain insight into the dynamics of the exponential growth of the novel coronavirus. Worst case, I'll simply have learned how to use Kepler.gl to visualize geospatial data, so the outcome should be positive either way.

In [1]:
from datetime import date, timedelta
import json
from pathlib import Path

from keplergl import KeplerGl
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
%matplotlib inline

DATA_PATH = Path("../covid-19-simulation/data")
SIM_NAME = "SimulationResult(n_days=250,tr_day=50,sd_day=40)"
SIM_RES_PATH = DATA_PATH / "results" / "sim-30km" / SIM_NAME
 # First infection in The Netherlands minus incubation time:
START_DATE = date(2020, 2, 27) - timedelta(days=5) 
KEPLER_CONF = "kepler-config.json"

### Load population data
This is the same data as in the previous post, taken from Statistics Netherlands.

In [2]:
df_pop = (gpd
          .read_file(DATA_PATH / "transformed" / "population.shp")
          .assign(geo_wkt=lambda df: df["geometry"].map(lambda geo: geo.to_wkt()))
          .assign(centroid=lambda df: df["geometry"].map(lambda geo: geo.centroid))
          .assign(lon=lambda df: df.centroid.map(lambda c: c.x),
                  lat=lambda df: df.centroid.map(lambda c: c.y)
          )
          [["geo_wkt", "lon", "lat", "pop"]]
)
df_pop.index.name = "hood_id"
df_pop.head(3)

,geo_wkt,lon,lat,pop
hood_id,,,,
0,POLYGON ((4.2874663832711484 52.11844854220218...,4.292010,52.112078,7955
1,POLYGON ((4.3000017436689992 52.09899194050742...,4.303211,52.104034,1855
2,POLYGON ((4.3273890785303832 52.09565737879255...,4.321339,52.097413,13320


### Load simulation data

In [3]:
column_rename_map = {
    "PersonState.untouched": "susceptible",  # To be more in line with SIR models.
    "PersonState.infected": "infected",
    "PersonState.recovered": "recovered",
    "PersonState.deceased": "deceased"
}

In [4]:
daily_count_dfs = []

for day, path_daily_count_csv in enumerate(sorted(SIM_RES_PATH.glob("daily_counts*"))):
    date = START_DATE + timedelta(days=day)

    df_dc = (
        pd.read_csv(path_daily_count_csv)
        .assign(date=date)
        .set_index(["date", "hood_id"])
        .rename(columns=column_rename_map)
        .applymap(int)
    )
    
    daily_count_dfs.append(df_dc)
    
df_daily_counts = pd.concat(daily_count_dfs).sort_index()

In [5]:
pd.concat((df_daily_counts.head(3), df_daily_counts.tail(3)))

susceptible  infected  recovered  deceased
date       hood_id                                            
2020-02-22 85             12030         0          0         0
           86              9200         0          0         0
           87              9840         0          0         0
2020-10-28 2717             785         0          0         0
           2740            1654        44       3770        97
           2741             740         6       1754        45

Now let's just denormalize the data a bit to make it easier to work with in Kepler.gl. In other words, add `lon` and `lat` to every row in the daily count DataFrame:

In [6]:
if "lon" not in df_daily_counts.columns:
    df_daily_counts = df_daily_counts.merge(df_pop, left_index=True, right_index=True, how="inner")
df_daily_counts.head(3)

susceptible  infected  recovered  deceased  \
date       hood_id                                               
2020-02-22 85             12030         0          0         0   
           86              9200         0          0         0   
           87              9840         0          0         0   

                                                              geo_wkt  \
date       hood_id                                                      
2020-02-22 85       POLYGON ((4.7437654333167867 52.26983760839533...   
           86       POLYGON ((4.7550087572913968 52.25162737834186...   
           87       POLYGON ((4.7680638414593837 52.27023804377086...   

                         lon        lat    pop  
date       hood_id                              
2020-02-22 85       4.736807  52.253185  12030  
           86       4.739852  52.237020   9200  
           87       4.795393  52.282207   9840

Need to do a bit of hacking in order to make Kepler normalize across the entire time series.


In [7]:
# df_dummy = pd.DataFrame(data={
#     "date": df_daily_counts.index.get_level_values("date").unique(),
#     "hood_id": -1,
#     "lon": 4.47917,
#     "lat": 51.9225,
#     "infected": 20000,
# }).set_index(["date", "hood_id"])

# df_daily_counts_with_max = pd.concat((df_daily_counts, df_dummy)).sort_index()

In [8]:
# df_daily_counts_with_max

## Let's map it!

In [9]:
# df_daily_counts.loc[(slice("2020-02-22", "2020-03-31"), slice(None))]

In [10]:
df_daily_counts["infected_per_100000"] = (df_daily_counts["infected"] / df_daily_counts["pop"] * 100_000).round().astype(int)

In [11]:
df_dummy = pd.DataFrame(data={
    "geo_wkt": df_pop.geo_wkt.values[0],
    "date": df_daily_counts.index.get_level_values("date").unique(),
    "hood_id": -1,
    "infected_per_100000": df_daily_counts["infected_per_100000"].max(),
}).set_index(["date", "hood_id"])

df_daily_counts_with_max = pd.concat((df_daily_counts, df_dummy)).sort_index()

In [12]:
df_daily_counts_with_max[["infected_per_100000", "geo_wkt"]].loc["2020-02-22"].reset_index().drop("hood_id", axis=1)

,date,infected_per_100000,geo_wkt
0,2020-02-22,23951,POLYGON ((4.2874663832711484 52.11844854220218...
1,2020-02-22,0,POLYGON ((4.7437654333167867 52.26983760839533...
2,2020-02-22,0,POLYGON ((4.7550087572913968 52.25162737834186...
3,2020-02-22,0,POLYGON ((4.7680638414593837 52.27023804377086...
4,2020-02-22,16,POLYGON ((4.8547199028694727 52.57189121752016...
...,...,...,...
283,2020-02-22,0,POLYGON ((4.5355870194703289 52.37461569226385...
284,2020-02-22,0,POLYGON ((4.5441197794007913 52.36909779109678...
285,2020-02-22,0,POLYGON ((4.9688865922883432 52.54089311574183...
286,2020-02-22,0,POLYGON ((4.4941803868280257 52.13021911541665...


In [13]:
with open(KEPLER_CONF, "r") as fp:
    config = json.load(fp)
    
nl_map = KeplerGl(height=800, data={"counts": 
                                    #df_daily_counts_with_max.reset_index()
                                    df_daily_counts_with_max[["infected_per_100000", "geo_wkt"]].loc["2020-02-22"]
#                                     df_daily_counts_with_max[["infected_per_100000", "geo_wkt"]].loc["2020-02-22"].reset_index().drop("hood_id", axis=1)
    #"counts": df_daily_counts.reset_index(),
#                                     "geo": df_pop.loc[df_pop.index.isin(df_daily_counts.index.get_level_values("hood_id"))].reset_index()
                                   },
                  config=config
                 )
nl_map

User Guide: https://github.com/keplergl/kepler.gl/blob/master/docs/keplergl-jupyter/user-guide.md


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'dd13jpd', 'type': …

In [15]:
with open(KEPLER_CONF, "w") as fp:
    json.dump(nl_map.config, fp)

In [14]:
for date in df_daily_counts.index.get_level_values("date").unique()[70:]:
    print(date)
    
    new_data = df_daily_counts_with_max[["infected_per_100000", "geo_wkt"]].loc[date]
    
    nl_map.add_data(new_data, name="counts")

2020-05-02 00:00:00
2020-05-03 00:00:00
2020-05-04 00:00:00
2020-05-05 00:00:00
2020-05-06 00:00:00
2020-05-07 00:00:00
2020-05-08 00:00:00
2020-05-09 00:00:00
2020-05-10 00:00:00
2020-05-11 00:00:00
2020-05-12 00:00:00
2020-05-13 00:00:00
2020-05-14 00:00:00
2020-05-15 00:00:00
2020-05-16 00:00:00
2020-05-17 00:00:00
2020-05-18 00:00:00
2020-05-19 00:00:00
2020-05-20 00:00:00
2020-05-21 00:00:00
2020-05-22 00:00:00
2020-05-23 00:00:00
2020-05-24 00:00:00
2020-05-25 00:00:00
2020-05-26 00:00:00
2020-05-27 00:00:00
2020-05-28 00:00:00
2020-05-29 00:00:00
2020-05-30 00:00:00
2020-05-31 00:00:00
2020-06-01 00:00:00
2020-06-02 00:00:00
2020-06-03 00:00:00
2020-06-04 00:00:00
2020-06-05 00:00:00
2020-06-06 00:00:00
2020-06-07 00:00:00
2020-06-08 00:00:00
2020-06-09 00:00:00
2020-06-10 00:00:00
2020-06-11 00:00:00
2020-06-12 00:00:00
2020-06-13 00:00:00
2020-06-14 00:00:00
2020-06-15 00:00:00
2020-06-16 00:00:00
2020-06-17 00:00:00
2020-06-18 00:00:00
2020-06-19 00:00:00
2020-06-20 00:00:00


### Lessons learned
- I had the Polygon data, but using it in Kepler.gl with time series data is a no-go. The Polygons have to be included for every time step, which really slows things down. Would be great to have to ability to just ship it once and then only provide updates using foreign key `hood_id`. 


### Caveats
Blabla chloropeth maps